In [ ]:
import os, pytz, folium, pandas as pd, numpy as np
from sqlalchemy import create_engine
from datetime import date, datetime, timedelta
import matplotlib.pyplot as plt, seaborn as sns

## parameter

In [ ]:
yeari, yearf = '2024', '2024'
weeki, weekf = '18', '31'

In [ ]:
di = datetime.strptime(f'{yeari}-{weeki}-1', "%Y-%W-%w").date()
df = datetime.strptime(f'{yearf}-{weekf}-1', "%Y-%W-%w").date() + timedelta(6)
ds = [di+timedelta(dt) for dt in range((df-di).days+1)]
daylist = ds
print(di, 'until', df)

In [ ]:
cdef = 'tl7_10m'# 'tl5_10m' 'tl6_10m' 'tl7_10m' 'tl8_10m' 'tl8_60m'
cdef_alt = '16m_10min'# tl5: 62 ... tl7: 16   tl8: 8

## database connection

In [ ]:
# database credentials
db_usr, db_pwd = os.getenv('DB_USR'), os.getenv('DB_PWD') # your database user name and password
# database login
host, port, db = 'nc-health-data-prod.cluster-ccsgl7rk4urn.eu-central-1.rds.amazonaws.com', 5432, 'master'

In [ ]:
# for queries with output
engine = create_engine('postgresql://'+db_usr+':'+db_pwd+'@'+host+':'+str(port)+'/'+db)
conn = engine.connect()

In [ ]:
conn.close()

## load contacts ...

### ... for cities

In [ ]:
ld_cn = f"""
            SELECT "day"
                , stime
                , dids
                , st_x(st_transform(geopoint, 4326)) as lon
                , st_y(st_transform(geopoint, 4326)) as lat
                , tl{cdef[2]}
                --, sources
    			--, in_stadium
                --, area_id
                , city
            FROM (
                SELECT cnv.*, t2p.*
                FROM covid_network_sdkv6_tl7_10m cnv 
                JOIN tile2plz t2p ON cnv.tile_id = t2p.tile_id
                WHERE "day" between '{str(di)}' and '{str(df)}'
            ) t1
            JOIN covid_plz_populate cpp ON t1.plz = cpp.id_plz5 
            WHERE city in ('Berlin','Dortmund','Düsseldorf','Frankfurt am Main','Gelsenkirchen','Hamburg','Köln','Leipzig','München','Stuttgart') 
        """

In [ ]:
ld_cn = f"""
    with cities1 as (
    	select osm_id, "name", way_area, way
    	from planet_osm_polygon
    	where osm_id in (-62428,-62422,-62649,-62400,-1829065,-2793104,-62578,-62539,-62782,-62522)
    ),
    cities2 as (
    	select osm_id, "name", max(way_area) as way_area
    	from cities1
    	group by 1,2
    ),
    cities3 as (
    	select c1."name", c1.way
    	from cities1 as c1
    	join cities2 as c2 on c1.way_area = c2.way_area
    ),
    cn_tmp as (
    	select
                  "day"
                , tl{cdef[2]}
                , stime
                , dids
                , sources
     			, bool_or(u.dist_stad < csa.radius_in_meter) as in_stadium
                , min(u.area_id) as area_id
     			, geopoint
            from covid_network_sdkv6_{cdef}, unnest(area_ids, dist_stads) u(area_id, dist_stad)
            left join cluster_search_areas_v2 csa on csa.area_id = u.area_id
            where
                    "day" between '{str(di-timedelta(1))}' and '{str(df)}'
            group by 1,2,3,4,5,8
    ),
    cn as (
        select
                  "day"
                , stime
                , dids
                , st_x(st_transform(geopoint, 4326)) as lon
                , st_y(st_transform(geopoint, 4326)) as lat
                , tl{cdef[2]}
                , sources
    			, in_stadium
                , area_id
                , "name" as city
        from cn_tmp
        join cities3 as c3 on st_contains(c3.way, cn_tmp.geopoint)
    )
    select *
    from cn
"""

In [ ]:
clist = pd.DataFrame(pd.read_sql_query(ld_cn, conn))

In [ ]:
clist.to_csv('output/09_contacts_cities_stadiums_tmp.csv', index=False)

In [ ]:
clist = pd.read_csv('output/09_contacts_cities_stadiums_tmp.csv')

In [ ]:
data = clist.copy(deep=True)
data['day'] = [d.date() for d in pd.to_datetime(data.day)]
data['stime'] = [d for d in pd.to_datetime(data.stime)]
data['dids'] = data.dids.apply(lambda x: x[2:-2].split("', '"))
data['sources'] = data.sources.apply(lambda x: x[1:-1].split(','))

In [ ]:
data['stime'] = data.stime.apply(lambda x: x.astimezone(pytz.timezone('Europe/Berlin')))
data['day'] = data.stime.apply(lambda x: x.date())
data = data[(data.day >= di) & (data.day <= df)]

In [ ]:
#data = data.explode(['dids']).reset_index(drop=True)
data = data.explode(['dids','sources']).reset_index(drop=True)
data = data.drop_duplicates()
#data = data.merge(data.drop(columns=['lon','lat','city']), on=['day','stime','tl7',])
data = data.merge(data.drop(columns=['lon','lat','in_stadium','area_id','city']), on=['day','stime','tl7',])#,'tl7','inside_building','lon','lat'])
data = data[data.dids_x != data.dids_y]
data['in_stadium'] = data.in_stadium.fillna(False)
data

In [ ]:
data_true = data[data.in_stadium==True]
data_false = data[data.in_stadium==False]
data_true = data_true[(data_true.sources_x=='GPS') | (data_true.sources_y=='GPS')]
data = pd.concat([data_true, data_false])

In [ ]:
#data = data[(data.sources_x=='GPS') | (data.sources_y=='GPS')]
pairs = []
for did1, did2 in zip(data.dids_x, data.dids_y):
    pair = f'{did1}_{did2}' if did1 < did2 else f'{did2}_{did1}'
    #print(did1, did2, pair)
    pairs.append(pair)
data.loc[:,'pair'] = pairs
#data = data.drop(columns=['dids_x','dids_y','tl7'])
data = data.drop(columns=['dids_x','dids_y','tl7','sources_x','sources_y'])
data = data.drop_duplicates()
dmin = data.day.min()
#data['tt'] = data.day.apply(lambda d: (d-dmin).days)*24 + data.stime.dt.hour
data['tt'] = data.day.apply(lambda d: (d-dmin).days)*24*6 + data.stime.dt.hour*6 + (data.stime.dt.hour//10)
print(data.tt.max(), ((data.day.max()-dmin).days+1)*24, ((data.day.max()-dmin).days+1)*720)
data

In [ ]:
data.to_csv('output/09_contacts_cities_stadiums.csv', index=False)

## contact heatmaps for host cities and host stadiums

In [ ]:
data = pd.read_csv('output/09_contacts_cities_stadiums.csv')
data['day'] = [d.date() for d in pd.to_datetime(data.day)]
data['stime'] = pd.to_datetime(data.stime)
data['hour'] = data.stime.dt.hour

In [ ]:
#data['hour'] = data.stime.dt.hour
#for_heatmap = data.groupby(['city','day','hour']).pair.apply(lambda x: len(set(x))).reset_index()
for_heatmap = data.groupby(['city','in_stadium','day','hour']).pair.apply(lambda x: len(set(x))).reset_index()
c1 = pd.DataFrame(set(for_heatmap.city), columns=['city'])
c2 = pd.DataFrame(set(for_heatmap.in_stadium), columns=['in_stadium'])
c3 = pd.DataFrame([d.date() for d in pd.date_range(di, df)], columns=['day'])
c4 = pd.DataFrame(list(range(24)), columns=['hour'])
#for_heatmap = c1.merge(c3, how='cross').merge(c4, how='cross').merge(for_heatmap, on=['city','day','hour'], how='left')
for_heatmap = c1.merge(c2, how='cross').merge(c3, how='cross').merge(c4, how='cross').merge(for_heatmap, on=['city','in_stadium','day','hour'], how='left')
for_heatmap

In [ ]:
sns.set_theme(style="ticks")

# Function to plot a heatmap inside each facet
def heatmap_func(data, **kwargs):
    #print(data.in_stadium.iloc[0])
    pivoted = data.pivot(index="hour", columns="day", values="pair")  # Reshape
    sns.heatmap(pivoted.loc[::-1], cmap="coolwarm", cbar=False, robust=~data.in_stadium.iloc[0], **kwargs)#, vmin=0., vmax=pivoted.max().max()

# Create a FacetGrid, grouping by 'category'
g = sns.FacetGrid(for_heatmap[for_heatmap.city.isin(['Berlin','Dortmund','Düsseldorf','Frankfurt am Main','Gelsenkirchen'])].fillna(0),
                  row="city", col="in_stadium", margin_titles=True, height=3, aspect=2.,
                  row_order=sorted(set(for_heatmap.city))[:5], col_order=[False, True])

# Map the custom heatmap function to each facet
g.map_dataframe(heatmap_func)

# Set yticks for all subplots
g.set(yticks=[24-x-.5 for x in range(0,24,2)], xticks=[x+.5 for x in range(len(set(for_heatmap.day)))])
g.set_yticklabels(list(range(0,24,2)), rotation=0)
g.set_xticklabels([str(d.month).zfill(2)+'/'+str(d.day).zfill(2) if d.weekday()==6 and d.isocalendar().week%2==0 else '' for d in sorted(set(for_heatmap.day))], rotation=0)

for ax in g.axes.flat:
    #ax.set_title(ax.get_title().split('=')[1][1:])
    ax.tick_params(axis="x", labelbottom=True)

# Modify margin titles
g.set_titles(row_template="{row_name}", col_template="{col_name}")#, size=14, fontweight='bold')
# Modify column titles
new_titles = {False: "city", True: "stadium"}
for ax, title in zip(g.axes.flat, g.col_names):  # g.col_names has the original column values
    ax.set_title(new_titles.get(title, title))  # Replace title if in dictionary

g.tight_layout()
plt.savefig(f'plots/fig5_{cdef_alt}/09_contacts_cities_stadiums_1.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/fig5_{cdef_alt}/09_contacts_cities_stadiums_1.pdf', bbox_inches='tight')
plt.show()

In [ ]:
sns.set_theme(style="ticks")

# Function to plot a heatmap inside each facet
def heatmap_func(data, **kwargs):
    #print(data.in_stadium.iloc[0])
    pivoted = data.pivot(index="hour", columns="day", values="pair")  # Reshape
    sns.heatmap(pivoted.loc[::-1], cmap="coolwarm", cbar=False, robust=~data.in_stadium.iloc[0], **kwargs)#, vmin=0., vmax=pivoted.max().max()

# Create a FacetGrid, grouping by 'category'
g = sns.FacetGrid(for_heatmap[for_heatmap.city.isin(['Hamburg','Köln','Leipzig','München','Stuttgart'])].fillna(0),
                  row="city", col="in_stadium", margin_titles=True, height=3, aspect=2.,
                  row_order=sorted(set(for_heatmap.city))[5:], col_order=[False, True])

# Map the custom heatmap function to each facet
g.map_dataframe(heatmap_func)

# Set yticks for all subplots
g.set(yticks=[24-x-.5 for x in range(0,24,2)], xticks=[x+.5 for x in range(len(set(for_heatmap.day)))])
g.set_yticklabels(list(range(0,24,2)), rotation=0)
g.set_xticklabels([str(d.month).zfill(2)+'/'+str(d.day).zfill(2) if d.weekday()==6 and d.isocalendar().week%2==0 else '' for d in sorted(set(for_heatmap.day))], rotation=0)

for ax in g.axes.flat:
    #ax.set_title(ax.get_title().split('=')[1][1:])
    ax.tick_params(axis="x", labelbottom=True)

# Modify margin titles
g.set_titles(row_template="{row_name}", col_template="{col_name}")#, size=14, fontweight='bold')
# Modify column titles
new_titles = {False: "city", True: "stadium"}
for ax, title in zip(g.axes.flat, g.col_names):  # g.col_names has the original column values
    ax.set_title(new_titles.get(title, title))  # Replace title if in dictionary

g.tight_layout()
plt.savefig(f'plots/fig5_{cdef_alt}/09_contacts_cities_stadiums_2.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/fig5_{cdef_alt}/09_contacts_cities_stadiums_2.pdf', bbox_inches='tight')
plt.show()

## activity around EURO 2024 matches

In [ ]:
# Creating the DataFrame with the updated match information, German city names, and times
# Creating the DataFrame with the updated match information, German city names, and times
data_group_a = {
    'City': ['München', 'Köln', 'Stuttgart', 'Köln', 'Frankfurt am Main', 'Stuttgart'],
    'Date': ['2024-06-14', '2024-06-15', '2024-06-19', '2024-06-19', '2024-06-23', '2024-06-23'],
    'Time': ['21:00', '15:00', '18:00', '21:00', '21:00', '21:00'],
    'Population': [1471508, 1085664, 634830, 1085664, 753056, 634830],
    'Match': [
        'Germany 5 - 1 Scotland', 
        'Hungary 1 - 3 Switzerland', 
        'Germany 2 - 0 Hungary', 
        'Scotland 1 - 1 Switzerland', 
        'Switzerland 1 - 1 Germany', 
        'Scotland 0 - 1 Hungary'
    ]
}
df_group_a = pd.DataFrame(data_group_a)

data_group_b = {
    'City': ['Berlin', 'Dortmund', 'Hamburg', 'Gelsenkirchen', 'Leipzig', 'Düsseldorf'],
    'Date': ['2024-06-15', '2024-06-15', '2024-06-19', '2024-06-20', '2024-06-24', '2024-06-24'],
    'Time': ['18:00', '21:00', '15:00', '21:00', '21:00', '21:00'],
    'Population': [3769000, 588462, 1841179, 260000, 597493, 619294],
    'Match': [
        'Spain 3 - 0 Croatia', 
        'Italy 2 - 1 Albania', 
        'Croatia 2 - 2 Albania', 
        'Spain 1 - 0 Italy', 
        'Croatia 1 - 1 Italy', 
        'Albania 0 - 1 Spain'
    ]
}
df_group_b = pd.DataFrame(data_group_b)

data_group_c = {
    'City': ['Stuttgart', 'Gelsenkirchen', 'München', 'Frankfurt am Main', 'Köln', 'München'],
    'Date': ['2024-06-16', '2024-06-16', '2024-06-20', '2024-06-20', '2024-06-25', '2024-06-25'],
    'Time': ['18:00', '21:00', '15:00', '18:00', '21:00', '21:00'],
    'Population': [634830, 260000, 1472000, 753056, 1085664, 1472000],
    'Match': [
        'Slovenia 1 - 1 Denmark', 
        'Serbia 0 - 1 England', 
        'Slovenia 1 - 1 Serbia', 
        'Denmark 1 - 1 England', 
        'England 0 - 0 Slovenia', 
        'Denmark 0 - 0 Serbia'
    ]
}

df_group_c= pd.DataFrame(data_group_c)

data_group_d = {
    'City': ['Hamburg', 'Düsseldorf', 'Berlin', 'Leipzig', 'Berlin', 'Dortmund'],
    'Date': ['2024-06-16', '2024-06-17', '2024-06-21', '2024-06-21', '2024-06-25', '2024-06-25'],
    'Time': ['15:00', '21:00', '18:00', '21:00', '18:00', '18:00'],
    'Population': [1841179, 619294, 3769000, 597493, 3769000, 588462],
    'Match': [
        'Poland 1 - 2 Netherlands', 
        'Austria 0 - 1 France', 
        'Poland 1 - 3 Austria', 
        'Netherlands 0 - 0 France', 
        'Netherlands 2 - 3 Austria', 
        'France 1 - 1 Poland'
    ]
}
df_group_d = pd.DataFrame(data_group_d)


data_group_e = {
    'City': ['München', 'Frankfurt am Main', 'Düsseldorf', 'Köln', 'Frankfurt am Main', 'Stuttgart'],
    'Date': ['2024-06-17', '2024-06-17', '2024-06-21', '2024-06-22', '2024-06-26', '2024-06-26'],
    'Time': ['15:00', '18:00', '15:00', '21:00', '18:00', '18:00'],
    'Population': [1488202, 753056, 617280, 1080394, 753056, 635911],
    'Match': [
        'Romania 3 - 0 Ukraine', 
        'Belgium 0 - 1 Slovakia', 
        'Slovakia 1 - 2 Ukraine', 
        'Belgium 2 - 0 Romania', 
        'Slovakia 1 - 1 Romania', 
        'Ukraine 0 - 0 Belgium'
    ]
}

df_group_e = pd.DataFrame(data_group_e)


# Creating the DataFrame with the match information, cities, dates, times, and populations
data_group_f = {
    'City': ['Dortmund', 'Leipzig', 'Hamburg', 'Dortmund', 'Gelsenkirchen', 'Hamburg'],
    'Date': ['2024-06-18', '2024-06-18', '2024-06-22', '2024-06-22', '2024-06-26', '2024-06-26'],
    'Time': ['18:00', '21:00', '15:00', '18:00', '21:00', '21:00'],
    'Population': [588462, 597493, 1841179, 588462, 260000, 1841179],
    'Match': [
        'Turkey 3 - 1 Georgia', 
        'Portugal 2 - 1 Czechia', 
        'Georgia 1 - 1 Czechia', 
        'Turkey 0 - 3 Portugal', 
        'Georgia 2 - 0 Portugal', 
        'Czechia 1 - 2 Turkey'
    ]
}
df_group_f = pd.DataFrame(data_group_f)




data_eurocup_16th = {
    'City': ['Dortmund', 'Berlin', 'Gelsenkirchen', 'Köln', 'Düsseldorf', 'Frankfurt am Main', 'München', 'Leipzig'],
    'Date': ['2024-06-29', '2024-06-29', '2024-06-30', '2024-06-30', '2024-07-01', '2024-07-01', '2024-07-02', '2024-07-02'],
    'Time': ['21:00', '18:00', '18:00', '21:00', '18:00', '21:00', '18:00', '21:00'],
    'Population': [588462, 3644826, 260000, 1085664, 617280, 736414, 1471508, 597493],
    'Match': [
        'Germany 2 - 0 Denmark',
        'Switzerland 2 - 0 Italy',
        'England 2 - 1 Slovakia ',
        'Spain 4 - 1 Georgia',
        'France 1 - 0 Belgium',
        'Portugal 0 - 0 Slovenia',
        'Romania 0 - 3 Netherlands',
        'Austria 1 - 2 Turkey'
    ]
}

df_eurocup_16th = pd.DataFrame(data_eurocup_16th)

# Creating the dataframe for the semifinals
data_semifinals = {
    'City': ['München', 'Dortmund'],
    'Date': ['2024-07-09', '2024-07-10'],
    'Time': ['21:00', '21:00'],
    'Population': [1471508, 588462],
    'Match': [
        'Spain 2 - 1 France',
        'Netherlands 1 - 2 England'
    ]
}

df_semifinals = pd.DataFrame(data_semifinals)

# Creating the dataframe for the quarterfinals
data_quarterfinals = {
    'City': ['Stuttgart', 'Hamburg', 'Berlin', 'Düsseldorf'],
    'Date': ['2024-07-05', '2024-07-05', '2024-07-06', '2024-07-06'],
    'Time': ['18:00', '21:00', '18:00', '21:00'],
    'Population': [634830, 1841179, 3644826, 617280],
    'Match': [
        'Spain 2 - 1 Germany',
        'Portugal 0 - 0 France ',
        'Netherlands 2 - 1 Turkey',
        'England 1 - 1 Switzerland'
    ]
}

df_quarterfinals = pd.DataFrame(data_quarterfinals)

data_final = {
    'City': ['Berlin'],
    'Date': ['2024-07-14'],
    'Time': ['21:00'],
    'Population': [3644826],
    'Match': [
        'Spain 2 - 1 England'
    ]
}

df_final = pd.DataFrame(data_final)



# Creating the DataFrame with the match information, cities, dates, times, and populations


df_group_a['Group'] = 'Group A'
df_group_b['Group'] = 'Group B'
df_group_c['Group'] = 'Group C'
df_group_d['Group'] = 'Group D'
df_group_e['Group'] = 'Group E'
df_group_f['Group'] = 'Group F'

# Combine the DataFrames
combined_df = pd.concat([df_group_a, df_group_b, df_group_c, df_group_d, df_group_e, df_group_f, df_eurocup_16th, df_quarterfinals, df_semifinals, df_final ], ignore_index=True)
combined_df

In [ ]:
combined_df = combined_df.rename(columns={'City':'city','Date':'day','Time':'hour_match','Match':'match'})
combined_df = combined_df[['city','day','hour_match','match']]
combined_df['hour_match'] = combined_df.hour_match.apply(lambda x: int(x.split(':')[0]))
combined_df['day'] = combined_df.day.apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date())
combined_df

In [ ]:
combined_df = combined_df.merge(for_heatmap.groupby(['city','day','hour']).pair.sum().reset_index(), on=['city','day'])
combined_df['hour_rel'] = combined_df.hour - combined_df.hour_match
#combined_df = combined_df.drop(columns=['hour_match','hour'])
combined_df['germany'] = combined_df.match.apply(lambda x: 'Germany' in x)
combined_df

In [ ]:
#baseline = for_heatmap.groupby(['hour']).pair.sum().reset_index().rename(columns={'pair':'baseline'})
baseline = for_heatmap[for_heatmap.in_stadium==False].groupby(['hour']).pair.sum().reset_index().rename(columns={'pair':'baseline'})
baseline['baseline'] = baseline.baseline / baseline.baseline.sum() * 24
baseline

In [ ]:
baseline = combined_df.groupby(['city','day']).pair.apply(np.sum).reset_index().rename(columns={'pair':'baseline'})
baseline

In [ ]:
#combined_df = combined_df.merge(baseline, on='hour')
combined_df = combined_df.merge(baseline, on=['city','day'])
combined_df['normed'] = combined_df.pair / combined_df.baseline

In [ ]:
normed_df = combined_df[(combined_df.hour_rel>=-7) & (combined_df.hour_rel<=5)]

In [ ]:
sns.set_theme(style="ticks")

# Initialize the figure with a logarithmic x axis
f, ax = plt.subplots(figsize=(9, 4))
#ax.set_xscale("log")

# Plot the orbital period with horizontal boxes
yup = .26
ax.fill_between([6.5,8.5], [yup,yup], color='gray', zorder=0, alpha=.5)
sns.boxplot(
    combined_df[(combined_df.hour_rel>=-7) & (combined_df.hour_rel<=5)], x="hour_rel", y="normed", hue="germany",
    whis=[5, 95], width=.6, palette=sns.color_palette(),#sns.color_palette("husl", 2)#"vlag"
)

# Add in points to show each observation
#sns.stripplot(planets, x="distance", y="method", size=4, color=".3")

# Tweak the visual presentation
ax.yaxis.grid(True)
ax.set(xlabel="time relative to match [hours]")
ax.set(ylabel="hourly contact share in host city")
sns.despine(trim=True, bottom=True, left=True)

# Get the legend object
legend = ax.get_legend()
# Change legend title
legend.set_title("German participation")
# Change legend labels
new_labels = ["no", "yes"]
for text, new_label in zip(legend.texts, new_labels):
    text.set_text(new_label)

plt.savefig(f'plots/fig6_{cdef_alt}/09_contacts_around_euro24_matches.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/fig6_{cdef_alt}/09_contacts_around_euro24_matches.pdf', bbox_inches='tight')
plt.show()

In [ ]:
restr = combined_df[(combined_df.hour_rel>=-7) & (combined_df.hour_rel<=5)]
restr[restr.match=='Germany 5 - 1 Scotland'].pair

In [ ]:
restr[restr.match=='Germany 2 - 0 Hungary'].pair

In [ ]:
#toplot2 = data[(data.day==date(2024,6,19)) & (data.stime.dt.hour==15) & (data.city=='Stuttgart')]
toplot = data[(data.day==date(2024,6,19)) & (data.stime.dt.hour==15) & (data.city=='Stuttgart')]
toplot

In [ ]:
# Create a folium map centered around the first point
m = folium.Map(location=[toplot.lat.mean(), toplot.lon.mean()], zoom_start=12)

# Add markers for each coordinate
for pair, lat, lon in zip(toplot.pair, toplot.lat, toplot.lon):
    folium.Circle(location=[lat, lon], popup=f"{pair}, Lat: {lat}, Lon: {lon}", color='blue', radius=10).add_to(m)
#for pair, lat, lon in zip(toplot2.pair, toplot2.lat, toplot2.lon):
#    folium.Circle(location=[lat, lon], popup=f"{pair}, Lat: {lat}, Lon: {lon}", color='red', radius=5).add_to(m)

# Display map
m

In [ ]:
normed_df = combined_df[(combined_df.hour_rel>=-7) & (combined_df.hour_rel<=5)]

In [ ]:
sns.set_theme(style="ticks")

# Initialize the figure with a logarithmic x axis
f, ax = plt.subplots(figsize=(9, 4))
#ax.set_xscale("log")

# Plot the orbital period with horizontal boxes
yup = 200
ax.fill_between([6.5,8.5], [yup,yup], color='gray', zorder=0, alpha=.5)
sns.boxplot(
    combined_df[(combined_df.hour_rel>=-7) & (combined_df.hour_rel<=5)], x="hour_rel", y="pair", hue="germany",
    whis=[5, 95], width=.6, palette=sns.color_palette(),#sns.color_palette("husl", 2)#"vlag"
)

# Add in points to show each observation
#sns.stripplot(planets, x="distance", y="method", size=4, color=".3")

# Tweak the visual presentation
ax.yaxis.grid(True)
ax.set(xlabel="time relative to match [hours]")
ax.set(ylabel="detected contacts in host city")
sns.despine(trim=True, bottom=True, left=True)

# Get the legend object
legend = ax.get_legend()
# Change legend title
legend.set_title("German participation")
# Change legend labels
new_labels = ["no", "yes"]
for text, new_label in zip(legend.texts, new_labels):
    text.set_text(new_label)

#plt.savefig(f'plots/fig6_{cdef_alt}/09_contacts_around_euro24_matches.jpg', bbox_inches='tight', dpi=300)
#plt.savefig(f'plots/fig6_{cdef_alt}/09_contacts_around_euro24_matches.pdf', bbox_inches='tight')
plt.show()